# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [65]:
# 1) Seu código aqui

url_pvl_munrj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url_pvl_munrj)
r.status_code
data_r = r.json()
df_r = pd.DataFrame(data_r['items'])
df_r.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,28822,Município,Resende,3304201,RJ,PVL02.001389/2018-28,Deferido,17944.105556/2018-71,2018-07-10T20:20:13Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,7000000.0,0,1,12/07/2018
1,38893,Município,Paty do Alferes,3303856,RJ,PVL02.007600/2019-05,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,3512000.0,0,1,30/10/2019


In [28]:
# 2) Seu código aqui

df_r.status.value_counts()

Deferido                                                 93
Arquivado                                                53
Regularizado                                             25
Arquivado por decurso de prazo                           23
Deferido (PVL-IF)                                        16
Encaminhado à PGFN com manifestação técnica favorável    13
Arquivado a pedido                                        6
Em retificação pelo interessado                           4
Em retificação pelo interessado (PVL-IF)                  1
Pendente de regularização                                 1
Name: status, dtype: int64

In [118]:
# 3) Seu código aqui

df_r_copy = df_r.copy()
df_r_copy['ano_status'] = df_r_copy['data_status'].str.slice(start=-4)
df_r_copy['ano_status'].info

<bound method Series.info of 0      2018
1      2019
2      2019
3      2019
4      2022
       ... 
230    2019
231    2011
232    2010
233    2009
234    2008
Name: ano_status, Length: 235, dtype: object>

In [46]:
# 4) Seu código aqui

df_r_copy['ano_status'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2022    14
2012    13
2011    10
2015     9
2009     9
2021     5
2017     5
2006     4
2002     4
2016     4
2018     4
2004     1
2003     1
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [94]:
#1) Seu código aqui

def chama_df_UF(UF, tipo_interessado):
    url_pvl_uf = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={UF}&tipo_interessado={tipo_interessado}'
    r = requests.get(url_pvl_uf)
    r.status_code
    data_r = r.json()
    df_r = pd.DataFrame(data_r['items'])
    return df_r

In [97]:
# 2) Seu código aqui

chama_df_UF('MG', 'Estado').status.value_counts()

Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado pela STN                                        3
Arquivado                                                 3
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [104]:
# 3) Seu código aqui

val = chama_df_UF('BA', 'Município').status=='Deferido'
chama_df_UF('BA', 'Município').interessado[val].value_counts()

Luís Eduardo Magalhães    14
Vitória da Conquista      11
Camaçari                  10
Lauro de Freitas          10
Barreiras                  9
                          ..
Coronel João Sá            1
São Sebastião do Passé     1
Sobradinho                 1
Alcobaça                   1
Barra                      1
Name: interessado, Length: 175, dtype: int64

In [112]:
# 4) Seu código aqui

chama_df_UF('BA', 'Estado').to_csv('solicitacoes_estado_bahia.csv', index=False)